In [ ]:
import json
from langchain_community.llms import Ollama
from typing import Dict

class LangChainSpeechEvaluator:
    def __init__(self, model="llama3"):
        self.llm = Ollama(model=model, temperature=0.3)

    def evaluate_speech(
        self,
        speaker: str,
        content: str,
        self_name: str,
        suspicion: Dict[str, float],
    ) -> Dict[str, str]:

        prompt = f"""
You are a player in a one-night werewolf game.

You are {self_name}.
Current suspicion levels: {suspicion}

You heard the following speech:
Speaker: {speaker}
Content: "{content}"

Answer ONLY in JSON:
{{"suspicion_delta": "increase" | "decrease" | "none"}}
"""

        response = self.llm.invoke(prompt)

        try:
            return json.loads(response)
        except Exception:
            return {"suspicion_delta": "none"}


In [ ]:
def make_handle_event(llm): 
    def handle_event(state: PlayerState) -> PlayerState:
        event = state["input"].get("event")
        if not event:
            return state

        memory = state["memory"]
        memory["history"].append(event)

        if event["type"] == "speech":
            speaker = event["speaker"]
            content = event["content"]

            # LLM に「評価」だけさせる
            judgment = llm.evaluate_speech(
                speaker=speaker,
                content=content,
                self_name=memory["self_name"],
                suspicion=memory["suspicion"],
            )

            delta = judgment.get("suspicion_delta")

            if delta == "increase":
                memory["suspicion"][speaker] = memory["suspicion"].get(speaker, 0.0) + 0.2
            elif delta == "decrease":
                memory["suspicion"][speaker] = max(
                    0.0, memory["suspicion"].get(speaker, 0.0) - 0.2
                )
            else:
                memory["suspicion"].setdefault(speaker, 0.1)

        return state
